In [23]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import cv2

In [24]:
%matplotlib inline
sys.path.append("..")

In [25]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [26]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [27]:
MODEL_NAME = 'final_inference_graph'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('final_training', 'label_map.pbtxt')
NUM_CLASSES = 6

In [28]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT , 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [29]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [19]:
def load_image_to_numpy_array(image):
    (im_width, im_height) = image.size
    print(image.size)
    return np.array(image.getdata()).reshape((im_width, im_height,3)).astype(np.uint8), im_height, im_width

In [9]:
#For Individual Images
PATH_TO_TEST_IMAGES_DIR = "annotations/test"    
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 2)]

IMAGE_SIZE = (12,8)

In [30]:
out = cv2.VideoWriter('final_training/Video Files/demo_video_final_1.avi', cv2.VideoWriter_fourcc(
        'M', 'J', 'P', 'G'), 10, (1624, 750))

In [31]:
def nms_tf(bboxes,psocres,threshold):
    '''
    NMS: NMS using in-built tf.image.non_max_suppression(bboxes,scores,top_n_proposal_after_nms,iou_threshould)
        
    Input:
        bboxes(tensor of bounding proposals) : Bounding Box Proposals in the format (x_min,y_min,x_max,y_max)
        threshold(float): Overlapping threshold above which proposals will be discarded.
        
    Output:
        filtered_bboxes(numpy array) :selected bboxes for which IOU is less than threshold.
    '''
    
    #First we need to convert bbox format from (x_min,y_min,x_max,y_max) to (y_min, x_min, y_max, x_max) ..
    #because tf.image.non_max_suppression method expects in that form.
    #For this we can use tf.unstack and tf.stack
    bboxes = tf.cast(bboxes,dtype=tf.float32)
    x_min,y_min,x_max,y_max = tf.unstack(bboxes,axis=1)
    bboxes = tf.stack([y_min,x_min,y_max,x_max],axis=1)
    bbox_indices = tf.image.non_max_suppression(bboxes,psocres, 100, iou_threshold=threshold)
    filtered_bboxes = tf.gather(bboxes,bbox_indices)
    scores = tf.gather(psocres,bbox_indices)
    y_min,x_min,y_max,x_max = tf.unstack(filtered_bboxes,axis=1)
    filtered_bboxes = tf.stack([x_min,y_min,x_max,y_max],axis=1)
    
    
    return filtered_bboxes

In [33]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # for image_path in TEST_IMAGE_PATHS:
            # image = Image.open(image_path)
            # image_np = load_image_to_numpy_array(image)
            # image_np_expanded = np.expand_dims(image_np, axis=0)
#         all_tensor_nodes = [n.name for n in tf.get_default_graph().as_graph_def().node]
#         all_feature_extractors = [x for x in all_tensor_nodes if "Relu" in x]
#         print(all_tensor_nodes, len(all_tensor_nodes))
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
#         detection_features = detection_graph.get_tensor_by_name('detection_features:0')
        b1s = detection_boxes[0]
        s1s = detection_scores[0]
        c1s = detection_classes[0]
#         df1s = detection_features[0]

        selected_indices = tf.image.non_max_suppression(b1s, s1s, 100, iou_threshold = 0.5)

        bxs = tf.gather(b1s, selected_indices)
        cls = tf.gather(c1s, selected_indices)
        scrs = tf.gather(s1s, selected_indices)
#         features= tf.gather(df1s, selected_indices)
        cap = cv2.VideoCapture('/Users/krunaaltavkar/Downloads/CSCI 527 - Project Data/Test_Video_Files/new_frames_2/video_10%03d.jpg')
        while(cap.isOpened()):
            # Read the frame
            ret, frame = cap.read()
            if frame is None:
                break
            frame = frame.astype('uint8')
            # Recolor the frame. By default, OpenCV uses BGR color space.
            # This short blog post explains this better:
            # https://www.learnopencv.com/why-does-opencv-use-bgr-color-format/
            color_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
            image_np_expanded = np.expand_dims(color_frame, axis=0)
        
            (boxes, scores, classes, num_ds) = sess.run(
                [bxs, scrs, cls, num_detections],
                feed_dict = {image_tensor: image_np_expanded})
            
            vis_util.visualize_boxes_and_labels_on_image_array(
                color_frame,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates = True,
                line_thickness=8,
                min_score_thresh=.40)
        
            cv2.imshow('frame', color_frame)
            output_rgb = cv2.cvtColor(color_frame, cv2.COLOR_RGB2BGR)
            out.write(output_rgb)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        out.release()
        cap.release()
        cv2.destroyAllWindows()